## Step 1: Import Packages and Declare Constants

In [1]:
import boto3
import sagemaker
import datetime as dt
import pandas as pd

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
sklearn_processor_version="0.23-1"
model_package_group_name="ChurnModelPackageGroup"
pipeline_name= "ChurnModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

[04/21/25 21:59:46] INFO     Defaulting to only available Python version: py3                     ]8;id=262668;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=64113;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=197627;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=393712;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#534\534]8;;\

## Step 2: Generate Baseline Dataset

Baseline Data will be used as part of SageMaker Clarify Step to generate SHAP Values 

In [3]:
def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    ## Convert to datetime columns
    df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
    df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
    ## Drop Rows with null values
    df = df.dropna()
    ## Create Column which gives the days between the last order and the first order
    df["first_last_days_diff"] = (df['lastorder']-df['firstorder']).dt.days
    ## Create Column which gives the days between when the customer record was created and the first order
    df['created'] = pd.to_datetime(df['created'])
    df['created_first_days_diff']=(df['created']-df['firstorder']).dt.days
    ## Drop Columns
    df.drop(['custid','created','firstorder','lastorder'],axis=1,inplace=True)
    ## Apply one hot encoding on favday and city columns
    df = pd.get_dummies(df,prefix=['favday','city'],columns=['favday','city'])
    return df

In [4]:
baseline_data = preprocess_data("data/storedata_total.csv")
baseline_data.pop("retained")
baseline_sample = baseline_data.sample(frac=0.0002)

In [5]:
pd.DataFrame(baseline_sample).to_csv("data/baseline.csv",header=False,index=False)

## Step 3: Generate Batch Dataset

In [6]:
batch_data = preprocess_data("data/storedata_total.csv")
batch_data.pop("retained")
batch_sample = batch_data.sample(frac=0.2)

In [7]:
pd.DataFrame(batch_sample).to_csv("data/batch.csv",header=False,index=False)

## Step 4: Copy Data and Scripts to S3 Bucket

In [8]:
s3_client = boto3.resource('s3')
s3_client.Bucket(default_bucket).upload_file("data/storedata_total.csv","data/storedata_total.csv")
s3_client.Bucket(default_bucket).upload_file("data/batch.csv","data/batch/batch.csv")
s3_client.Bucket(default_bucket).upload_file("data/baseline.csv","input/baseline/baseline.csv")

In [9]:
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/preprocess.py","input/code/preprocess.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/evaluate.py","input/code/evaluate.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/generate_config.py","input/code/generate_config.py")

## Step 5: Get the Pipeline Instance

In [10]:
from pipelines.customerchurn.pipeline import get_pipeline

pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

[04/21/25 21:59:48] WARNING  The input argument instance_type of function                          ]8;id=933586;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=919151;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.retrieve) is a pipeline variable (<class                        
                             'sagemaker.workflow.parameters.ParameterString'>), which is                           
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    INFO     Defaulting to only available Python version: py3                     ]8;id=759310;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=842737;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    WARNING  The input argument instance_type of function                          ]8;id=673711;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=309731;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.retrieve) is a pipeline variable (<class                        
                             'sagemaker.workflow.parameters.ParameterString'>), which is                           
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=873600;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=541629;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    WARNING  The class JsonGet has been renamed in sagemaker>=2.                 ]8;id=868141;file:///opt/conda/lib/python3.12/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=119769;file:///opt/conda/lib/python3.12/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

In [11]:
pipeline.definition()

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=625938;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=174309;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  No finished training job found associated with this estimator.       ]8;id=602582;file:///opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=108508;file:///opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  No finished training job found associated with this estimator.       ]8;id=872936;file:///opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=974096;file:///opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  Popping out 'HyperParameterTuningJobName' from the pipeline           ]8;id=971177;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=885748;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             definition by default since it will be overridden at pipeline                         
                             execution time. Please utilize the PipelineDefinitionConfig to                        
                             persist this field in the pipeline definition if desired.                             

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=565719;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=334713;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

...

                    WARNING  Popping out 'ModelName' from the pipeline definition by default since ]8;id=107532;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=962388;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             it will be overridden at pipeline execution time. Please utilize the                  
                             PipelineDefinitionConfig to persist this field in the pipeline                        
                             definition if desired.                                                                

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=990464;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=703182;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'TransformJobName' from the pipeline definition by        ]8;id=340400;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=939390;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=973232;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=181788;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

'{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "ProcessingInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "TrainingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "InputData", "Type": "String", "DefaultValue": "s3://sagemaker-us-east-1-523271815992/data/storedata_total.csv"}, {"Name": "BatchData", "Type": "String", "DefaultValue": "s3://sagemaker-us-east-1-523271815992/data/batch/batch.csv"}], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"Get": "Execution.PipelineExecutionId"}}, "Steps": [{"Name": "ChurnModelProcess", "Type": "Processing", "Arguments": {"ProcessingResources": {"ClusterConfig": {"InstanceType": {"Get": "Parameters.ProcessingInstanceType"}, "InstanceCount": {"Get": "Parameters.ProcessingInstanceCount"}, "VolumeSizeInGB": 30}}, "AppSpecification": {"ImageUri": "683313688378.d

## Step 5: Submit the pipeline to SageMaker and start execution

In [12]:
pipeline.upsert(role_arn=role)

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=424151;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=539758;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  No finished training job found associated with this estimator.       ]8;id=12921;file:///opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=617517;file:///opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  No finished training job found associated with this estimator.       ]8;id=103096;file:///opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=702466;file:///opt/conda/lib/python3.12/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  Popping out 'HyperParameterTuningJobName' from the pipeline           ]8;id=562845;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=107588;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             definition by default since it will be overridden at pipeline                         
                             execution time. Please utilize the PipelineDefinitionConfig to                        
                             persist this field in the pipeline definition if desired.                             

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=588291;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=457469;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

...

                    WARNING  Popping out 'ModelName' from the pipeline definition by default since ]8;id=418647;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=280172;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             it will be overridden at pipeline execution time. Please utilize the                  
                             PipelineDefinitionConfig to persist this field in the pipeline                        
                             definition if desired.                                                                

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=508722;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=449225;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'TransformJobName' from the pipeline definition by        ]8;id=431369;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=633251;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=843685;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=867917;file:///opt/conda/lib/python3.12/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:523271815992:pipeline/ChurnModelSMPipeline',
 'ResponseMetadata': {'RequestId': 'd320ed6b-c96b-4145-a8db-2205f7efd30c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd320ed6b-c96b-4145-a8db-2205f7efd30c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Mon, 21 Apr 2025 21:59:49 GMT'},
  'RetryAttempts': 0}}

Start Pipeline Execution

In [13]:
execution = pipeline.start()

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [14]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:523271815992:pipeline/ChurnModelSMPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:523271815992:pipeline/ChurnModelSMPipeline/execution/zopqwyxwj0v6',
 'PipelineExecutionDisplayName': 'execution-1745272789293',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2025, 4, 21, 21, 59, 49, 234000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 4, 21, 21, 59, 49, 234000, tzinfo=tzlocal()),
 'CreatedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::523271815992:assumed-role/AmazonSageMaker-ExecutionRole-20250421T153819/SageMaker',
   'PrincipalId': 'AROAXTVLOIM4MXBP2TW6B:SageMaker'}},
 'LastModifiedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::523271815992:assumed-role/AmazonSageMaker-ExecutionRole-20250421T153819/SageMaker',
   'PrincipalId': 'AROAXTVLOIM4MXBP2TW6B:SageMaker'}},
 'ResponseMetadata': {'RequestId': 'f9463b8c-e81b-4810-807e-f7be8866425f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-a

We can list the execution steps to check out the status and artifacts:

In [16]:
execution.list_steps()

[{'StepName': 'ClarifyProcessingStep',
  'StartTime': datetime.datetime(2025, 4, 21, 22, 10, 33, 867000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 4, 21, 22, 20, 38, 581000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': 'ClientError: ClientError: An error occurred (ModelError) when calling the InvokeEndpoint operation (reached max retries: 0): Received client error (415) from primary with message "Loading csv data failed with Exception, please ensure data is in csv format:\n <class \'ValueError\'>\n could not convert string to float: \'False\'". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/sm-clarify-pipelines-zopqwyxwj0v6-ChurnCreateMo-1745273652-9dd2 in account 523271815992 for more information., exit code: 1',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:523271815992:processing-job/pipelines-zopqwyxwj0v6-ClarifyProcessingSte-3yIWVYqq02'}},
  'Attempt